<a href="https://colab.research.google.com/github/HayeonLee/Meta_Learning_Tutorial/blob/master/Meta_Learning_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Code: https://github.com/abdulfatir/prototypical-networks-tensorflow

# Few-shot Learning 1: Prototypical Network

이 튜토리얼에서 우리는 meta knowledge를 학습하여 few-shot learning을 수행하는 두가지 대표적인 네트워크, [Prototypical Network](https://arxiv.org/abs/1703.05175) (ProtoNet)과 [Model-Agnostic Meta-Learning for Fast Adaptation of Deep Networks](https://arxiv.org/abs/1703.03400) (MAML)을 배울 것입니다.





## Few-shot Learning이란?
(설명 추가 예정)

## Prototypical Network란?
(설명 추가 예정)
![대체 텍스트](https://drive.google.com/uc?id=1mdv1z3BlFzJm3SaAWW2hajs9eqoXU5qe)

## ProtoNet 튜토리얼

이제부터 어떻게 네트워크를 만들고 ProtoNet 학습 시킬지를 배워봅시다.
순서는 다음과 같습니다.
0. 데이터셋을 다운로드 받는다.
1. Tensorflow와 다른 library들을 불러온다.
2. 데이터셋을 전처리한다.
3. 모델을 만든다.
4. loss와 optimizer를 정의한다.
5. Training loop를 정의한다.
6. Training!
7. Test

그럼 하나씩 진행해보도록 합시다.



### Download Omniglot Dataset


In [0]:
!mkdir -p data/omniglot/data
!mkdir -p data/omniglot/split
!wget -O data/omniglot/split/trainval.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/trainval.txt?raw=true
!wget -O data/omniglot/split/train.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/train.txt?raw=true
!wget -O data/omniglot/split/test.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/test.txt?raw=true
!wget -O images_background.zip https://github.com/brendenlake/omniglot/blob/master/python/images_background.zip?raw=true
!wget -O images_evaluation.zip https://github.com/brendenlake/omniglot/blob/master/python/images_evaluation.zip?raw=true
!unzip images_background.zip -d data/omniglot/data
!unzip images_evaluation.zip -d data/omniglot/data
!mv data/omniglot/data/images_background/* data/omniglot/data/
!mv data/omniglot/data/images_evaluation/* data/omniglot/data/
!rmdir data/images_background
!rmdir data/images_evaluation

### Import Tensorflow and other libraries


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [3]:
#기존에 설치된 다른 버전의 tensorflow를 제거합니다.
!pip uninstall tensorboard -y
!pip uninstall tensorflow-gpu -y
!pip uninstall tensorflow -y

Uninstalling tensorboard-1.14.0:
  Successfully uninstalled tensorboard-1.14.0
Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0


In [0]:
!pip install tensorflow-gpu==1.14.0 #tensorflow gpu 버전을 설치합니다

In [0]:
import tensorflow as tf # tensorflow를 import해줍니다

In [6]:
tf.__version__ # 내가 사용할 tensorflow의 버전을 나타냅니다

'1.14.0'

In [7]:
if tf.test.gpu_device_name():
  print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
  print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [0]:
# 필요한 라이브러리를 import합니다.
%matplotlib inline
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import glob
import matplotlib.pyplot as plt

### Prepare the dataset

Omniglot 데이터셋을 이용하여 ProtoNet를 학습시켜봅시다.

학습이 끝나면 ProtoNet은 처음 보는 Task들에 대해서 few-shot classification을 잘 수행하게 됩니다.

주어진 정보들을 이용하여 빈 칸을 채워보세요! ([    ]가 빈 칸을 나타냅니다. 괄호를 지우고 알맞은 코드를 써주세요)

1. 매 epoch마다 100가지의 다른 에피소드가 주어집니다.
2. 매 에피소드마다 태스크의 분류해야할 클래스의 종류는 60개입니다.
3. 태스크의 각 클래스 별로 참고 할 수 있도록 주어진 이미지는 5개입니다.
4. 태스크의 각 클래스 별로 맞춰야할 이미지는 5개입니다.
5. 이미지는 흑백 이미지로 단일 채널입니다.


In [0]:
n_epochs = 20 # epoch 수
n_episodes = 100 # 1번
n_way = 60 # 2번
n_shot = 5 # 3번
n_query = 5 # 4번
n_examples = 20
im_width = 28 # 이미지 넓이
im_height = 28 # 이미지 높이
channels = 1 # 5번

In [20]:
# 훈련 데이터 셋을 불러옵니다.
root_dir = './data/omniglot' 
train_split_path = os.path.join(root_dir, 'split', 'train.txt') # 경로: ./data/omniglot/split/train.txt

# 모든 클래스의 경로를 train_classes 리스트에 저장합니다.
with open(train_split_path, 'r') as train_split:
    train_classes = [line.rstrip() for line in train_split.readlines()] 
n_classes = len(train_classes)

# 각 클래스별 모든 이미지의 경로를 불러옵니다.
train_dataset = np.zeros([n_classes, n_examples, im_height, im_width], dtype=np.float32)
for i, tc in enumerate(train_classes):
    alphabet, character, rotation = tc.split('/')
    rotation = float(rotation[3:])
    im_dir = os.path.join(root_dir, 'data', alphabet, character)
    im_files = sorted(glob.glob(os.path.join(im_dir, '*.png')))
    # 이미지를 불러들여 회전, 크기 변환, 형 변환 등을 수행한 뒤 train_dataset 행렬에 저장합니다.
    for j, im_file in enumerate(im_files):
        im = 1. - np.array(Image.open(im_file).rotate(rotation).resize((im_width, im_height)), np.float32, copy=False)
        train_dataset[i, j] = im
c, ni, w, h = train_dataset.shape
print('불러온 훈련 데이터 셋의 특징')
print('클래스 갯수: {}, 클래스 별 이미지 갯수: {}, 이미지 넓이: {}, 이미지 높이: {}'.format(c, ni, w, h))

클래스 갯수: 4112, 클래스 별 이미지 갯수: 20, 이미지 넓이: 28, 이미지 높이: 28


## Create the models
이제 Prototypical Network를 만들어봅시다!

우리가 만들 Prototypical Network의 구조는 아래 그림들과 같습니다.
![대체 텍스트](https://drive.google.com/uc?id=1nBP6VvWyDLb_kGmg3a_oEk4FP0oJSiXa)

빈 칸을 채워서 그림과 맞는 모델을 만들어보세요.

In [0]:
h_dim = 64 # hidden channels
z_dim = 64 # output channels

In [0]:
def conv_block(inputs, out_channels, name='conv'):
    with tf.variable_scope(name):
        conv = tf.layers.conv2d(inputs, out_channels, kernel_size=3, padding='SAME')
        conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True)
        conv = tf.nn.relu(conv)
        conv = tf.contrib.layers.max_pool2d(conv, 2)
        return conv

In [0]:
def encoder(x, h_dim, z_dim, reuse=False):
    with tf.variable_scope('encoder', reuse=reuse):
        net = conv_block(x, h_dim, name='Conv_Block_1')
        net = conv_block(net, h_dim, name='Conv_Block_2')
        net = conv_block(net, h_dim, name='Conv_Block_3')
        net = conv_block(net, z_dim, name='Conv_Block_4')
        net = tf.contrib.layers.flatten(net)
        return net

## Define metric function: Euclidean distance

\begin{equation*}
d(z, z') = ||z-z'||^2
\end{equation*}
\begin{equation*}
d(f_\phi(\mathbf{x}), \mathbf{c}_k)) = ||f_\phi(\mathbf{x})-\mathbf{c}_k||^2
\end{equation*}

In [0]:
# N개의 임베딩 벡터와 M개의 임베딩 벡터 간의 유클리디안 거리를 계산합니다.
def euclidean_distance(a, b):
    # a.shape = N x D
    # b.shape = M x D
    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)

## Set placeholders

In [0]:
supports = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
queries = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
support_shape = tf.shape(supports)
query_shape = tf.shape(queries)
num_classes, num_support = support_shape[0], support_shape[1]
num_queries = query_shape[1]
y = tf.placeholder(tf.int64, [None, None])
y_one_hot = tf.one_hot(y, depth=num_classes)

## Get prototypes and query embeddings

![대체 텍스트](https://drive.google.com/uc?id=1mN4CNa9AOq2nQjK8hvXN7KztZHwZoLDE)

Support 포인트들을 평균 내서 클래스별 프로토타입을 구하고 쿼리 포인트들은 인스턴스로 임베딩 됩니다.

In [0]:
# support 이미지를 임베딩 벡터로 인코딩합니다.
emb_supports = encoder(tf.reshape(supports, [num_classes * num_support, im_height, im_width, channels]), h_dim, z_dim)
emb_dim = tf.shape(emb_supports)[-1]
# 각 클래스의 support 임베딩들의 평균을 구하여 각 클래스별 프로토타입을 얻습니다.
prototypes = tf.reduce_mean(tf.reshape(emb_supports, [num_classes, num_support, emb_dim]), axis=1)
# 주어진 쿼리를 임베딩 벡터로 인코딩합니다.
emb_queries = encoder(tf.reshape(queries, [num_classes * num_queries, im_height, im_width, channels]), h_dim, z_dim, reuse=True)

## Get prototypical loss with Euclidean distance

![대체 텍스트](https://drive.google.com/uc?id=1LBkqCntsBNiP7RyaCazXopswhslabhJE)

In [0]:
# 쿼리 임베딩들과 프로토타입들 간의 유클리디안 거리를 계산합니다.
dists = euclidean_distance(emb_queries, prototypes)
# 
log_p_y = tf.reshape(tf.nn.log_softmax(-dists), [num_classes, num_queries, -1])
ce_loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, log_p_y), axis=-1), [-1]))
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(log_p_y, axis=-1), y)))

train_op = tf.train.AdamOptimizer().minimize(ce_loss)

## Set session

In [0]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

## Let's run the code!

In [0]:
for ep in range(n_epochs):
    for epi in range(n_episodes):
        epi_classes = np.random.permutation(n_classes)[:n_way]
        support = np.zeros([n_way, n_shot, im_height, im_width], dtype=np.float32)
        query = np.zeros([n_way, n_query, im_height, im_width], dtype=np.float32)
        for i, epi_cls in enumerate(epi_classes):
            selected = np.random.permutation(n_examples)[:n_shot + n_query]
            support[i] = train_dataset[epi_cls, selected[:n_shot]]
            query[i] = train_dataset[epi_cls, selected[n_shot:]]
        support = np.expand_dims(support, axis=-1)
        query = np.expand_dims(query, axis=-1)
        labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
        _, ls, ac = sess.run([train_op, ce_loss, acc], feed_dict={x: support, q: query, y:labels})
        if (epi+1) % 50 == 0:
            print('[epoch {}/{}, episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(ep+1, n_epochs, epi+1, n_episodes, ls, ac))

## Visualization

![대체 텍스트](https://drive.google.com/uc?id=1fx5bVyr-VTHjtW21bWGt-LeghmQ3Iaom)

In [0]:
# add code

## Test

In [0]:
# Load Test Dataset
root_dir = './data/omniglot'
test_split_path = os.path.join(root_dir, 'splits', 'test.txt')
with open(test_split_path, 'r') as test_split:
    test_classes = [line.rstrip() for line in test_split.readlines()]
n_test_classes = len(test_classes)
test_dataset = np.zeros([n_test_classes, n_examples, im_height, im_width], dtype=np.float32)
for i, tc in enumerate(test_classes):
    alphabet, character, rotation = tc.split('/')
    rotation = float(rotation[3:])
    im_dir = os.path.join(root_dir, 'data', alphabet, character)
    im_files = sorted(glob.glob(os.path.join(im_dir, '*.png')))
    for j, im_file in enumerate(im_files):
        im = 1. - np.array(Image.open(im_file).rotate(rotation).resize((im_width, im_height)), np.float32, copy=False)
        test_dataset[i, j] = im
print(test_dataset.shape)

In [0]:
n_test_episodes = 1000
n_test_way = 20
n_test_shot = 5
n_test_query = 15

In [0]:
print('Testing...')
avg_acc = 0.
for epi in range(n_test_episodes):
    epi_classes = np.random.permutation(n_test_classes)[:n_test_way]
    support = np.zeros([n_test_way, n_test_shot, im_height, im_width], dtype=np.float32)
    query = np.zeros([n_test_way, n_test_query, im_height, im_width], dtype=np.float32)
    for i, epi_cls in enumerate(epi_classes):
        selected = np.random.permutation(n_examples)[:n_test_shot + n_test_query]
        support[i] = test_dataset[epi_cls, selected[:n_test_shot]]
        query[i] = test_dataset[epi_cls, selected[n_test_shot:]]
    support = np.expand_dims(support, axis=-1)
    query = np.expand_dims(query, axis=-1)
    labels = np.tile(np.arange(n_test_way)[:, np.newaxis], (1, n_test_query)).astype(np.uint8)
    ls, ac = sess.run([ce_loss, acc], feed_dict={x: support, q: query, y:labels})
    avg_acc += ac
    if (epi+1) % 50 == 0:
        print('[test episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(epi+1, n_test_episodes, ls, ac))
avg_acc /= n_test_episodes
print('Average Test Accuracy: {:.5f}'.format(avg_acc))